In [2]:
import requests

def query_medlineplus(term):
    url = "https://wsearch.nlm.nih.gov/ws/query"
    params = {
        "db": "healthTopics",
        "term": term,
        "retmax": 5,
        "rettype": "brief",
        "format": "json"
    }

    response = requests.get(url, params=params)

    # Print raw response for debugging
    print("Status Code:", response.status_code)
    print("Response Text:", response.text)

    try:
        data = response.json()
        documents = data.get("list", {}).get("document", [])
        for doc in documents:
            print("Title:", doc.get("content", {}).get("title"))
            print("URL:", doc.get("url"))
            print()
    except Exception as e:
        print("Failed to decode JSON:", e)

# Try it with a basic term
query_medlineplus("diabetes")


Status Code: 200
Response Text: <?xml version="1.0" encoding="UTF-8"?>
<nlmSearchResult>
  <term>diabetes</term>
  <file>viv_LOX7yP</file>
  <server>pvlb7srch16</server>
  <count>285</count>
  <retstart>0</retstart>
  <retmax>5</retmax>
  <list num="285" start="0" per="5">
    <document rank="2" url="https://medlineplus.gov/diabetes.html">
      <content name="title">&lt;span class="qt0"&gt;Diabetes&lt;/span&gt;</content>
      <content name="organizationName">National Library of Medicine</content>
      <content name="altTitle">&lt;span class="qt0"&gt;Diabetes Mellitus&lt;/span&gt;</content>
      <content name="altTitle">Sugar &lt;span class="qt0"&gt;Diabetes&lt;/span&gt;</content>
      <content name="altTitle">DM</content>
      <content name="FullSummary">What is &lt;span class="qt0"&gt;diabetes&lt;/span&gt;?&lt;p&gt;&lt;span class="qt0"&gt;Diabetes&lt;/span&gt;, also known as &lt;span class="qt0"&gt;diabetes mellitus&lt;/span&gt;, is a disease in which your blood glucose, or bloo

In [5]:
import requests
import xml.etree.ElementTree as ET

def query_treatments_xml(term):
    url = "https://wsearch.nlm.nih.gov/ws/query"
    params = {
        "db": "healthTopics",
        "term": term,
        "retmax": 10,
        "rettype": "brief",
        "format": "xml"
    }
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Failed to fetch data:", response.status_code)
        return

    # Parse XML response
    root = ET.fromstring(response.content)

    found_treatments = False

    # The XML structure has <document> elements inside <list>
    for doc in root.findall(".//document"):
        title = doc.findtext("content/title")
        snippet = doc.findtext("content/snippet")
        url = doc.findtext("url")

        # Check if title or snippet contains "treatment" or "therapy"
        text_to_check = (title or "").lower() + " " + (snippet or "").lower()

        if "treatment" in text_to_check or "therapy" in text_to_check:
            found_treatments = True
            print(f"Title: {title}")
            print(f"URL: {url}")
            print()

    if not found_treatments:
        print(f"No treatment-related info found for: {term}")

# Try with "diabetes"
query_treatments_xml("covid")


No treatment-related info found for: covid


In [9]:
!pip install pymed


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import requests
import json
from pymed import PubMed

def get_disease_data(disease_name):
    # Base URL for the Clinical Table Search Service API
    base_url = "https://clinicaltables.nlm.nih.gov/api/conditions/v3/search"
    
    # Query parameters
    params = {
        "terms": disease_name,  # Search term (e.g., "diabetes")
        "df": "primary_name,terms,synonyms,icd10cm",  # Fields to retrieve
        "max_list": 10  # Limit results to 10 for simplicity
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the JSON response
        data = response.json()
        
        # Extract relevant fields
        total_results = data[0]
        condition_codes = data[1]
        extra_data = data[3]  # List of lists: [primary_name, terms, synonyms, icd10cm]
        
        print(f"Total results found for '{disease_name}': {total_results}")
        if total_results == 0:
            print("No results found.")
            return None
        
        # Store the first primary name for PubMed query
        first_primary_name = None
        
        # Display disease information
        for i, row in enumerate(extra_data):
            condition_data = {
                "Primary Name": row[0],  # primary_name
                "Terms": row[1],        # terms
                "Synonyms": row[2],     # synonyms
                "ICD-10-CM": row[3]     # icd10cm
            }
            if i == 0:  # Store the first primary name
                first_primary_name = row[0]
            print(f"\nResult {i+1}:")
            print(json.dumps(condition_data, indent=2))
        
        return first_primary_name
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def get_treatment_data(disease_name):
    # Get disease data and primary name
    primary_name = get_disease_data(disease_name)
    if not primary_name:
        print("Cannot fetch treatment data due to no results or error.")
        return
    
    # Query PubMed for treatment-related papers
    try:
        pubmed = PubMed(tool="TreatmentFinder", email="your_email@example.com")  # Replace with your email
        query = f"{primary_name} treatment"
        results = pubmed.query(query, max_results=3)
        
        print(f"\nTreatment-related papers for '{primary_name}':")
        for article in results:
            print(f"Title: {article.title}")
            print(f"Abstract: {article.abstract or 'No abstract available'}")
            print(f"DOI: {article.doi or 'No DOI available'}")
            print("-" * 50)
            
    except Exception as e:
        print(f"Error fetching PubMed data: {e}")

# Test with a disease
disease = "diabetes"
get_treatment_data(disease)

Total results found for 'diabetes': 21

Result 1:
{
  "Primary Name": "Diabetes mellitus",
  "Terms": "",
  "Synonyms": "DM",
  "ICD-10-CM": "{\"code\":\"E11.9\",\"name\":\"Type 2 diabetes mellitus without complications\"}"
}

Result 2:
{
  "Primary Name": "Diabetes mellitus - non insulin dependent",
  "Terms": "",
  "Synonyms": "NIDDM,diabetes mellitus type II",
  "ICD-10-CM": "{\"code\":\"E11.9\",\"name\":\"Type 2 diabetes mellitus without complications\"}"
}

Result 3:
{
  "Primary Name": "Diabetes mellitus - insulin dependent",
  "Terms": "",
  "Synonyms": "IDDM,diabetes mellitus type I",
  "ICD-10-CM": "{\"code\":\"E10.9\",\"name\":\"Type 1 diabetes mellitus without complications\"}"
}

Result 4:
{
  "Primary Name": "Diabetic ketoacidosis",
  "Terms": "",
  "Synonyms": "DKA,ketoacidos diabetic",
  "ICD-10-CM": "{\"code\":\"E13.10\",\"name\":\"Other specified diabetes mellitus with ketoacidosis without coma\"}"
}

Result 5:
{
  "Primary Name": "Neuropathy - diabetic",
  "Terms": ""

In [14]:
import requests
from pymed import PubMed
import re

def extract_treatment_names(abstract):
    # Keywords to identify treatment names
    treatment_keywords = [
        r"insulin", r"metformin", r"sglt2 inhibitor", r"glp-1 receptor agonist", 
        r"lifestyle", r"diet", r"exercise", r"surgery", r"stem cell therapy", 
        r"therapeutic art", r"crocin", r"sulfonylurea", r"dpp-4 inhibitor",
        r"chemotherapy", r"radiation", r"immunotherapy", r"antibiotic", 
        r"antiviral", r"steroid", r"vaccine"
    ]
    treatments = []
    if abstract:
        for keyword in treatment_keywords:
            matches = re.findall(keyword, abstract, re.IGNORECASE)
            for match in matches:
                if match and match.lower() not in [t.lower() for t in treatments]:
                    treatments.append(match)
    return treatments

def get_treatment_names(disease_name):
    try:
        pubmed = PubMed(tool="TreatmentFinder", email="your_email@example.com")  # Replace with your email
        query = f"{disease_name} treatment"
        results = pubmed.query(query, max_results=5)
        
        treatments_found = []
        for article in results:
            abstract = article.abstract or ""
            article_treatments = extract_treatment_names(abstract)
            treatments_found.extend(article_treatments)
        
        # Standard treatments for common diseases
        standard_treatments = []
        if "diabetes" in disease_name.lower():
            standard_treatments = [
                "Insulin",
                "Metformin",
                "SGLT2 inhibitors",
                "GLP-1 receptor agonists",
                "Lifestyle changes",
                "Sulfonylureas",
                "DPP-4 inhibitors"
            ]
        
        # Combine and deduplicate treatments
        all_treatments = list(set(treatments_found + standard_treatments))
        
        if not all_treatments:
            print(f"No treatments found for '{disease_name}'.")
            return
        
        print(f"Treatments for {disease_name}:")
        for i, treatment in enumerate(sorted(all_treatments), 1):
            print(f"{i}. {treatment}")
            
    except Exception as e:
        print(f"Error fetching PubMed data: {e}")
        if "diabetes" in disease_name.lower():
            print(f"Using standard treatments for {disease_name}:")
            for i, treatment in enumerate(sorted(standard_treatments), 1):
                print(f"{i}. {treatment}")

# Get disease name from user
disease_name = input("Enter the disease name: ")
get_treatment_names(disease_name)

Treatments for covid:
1. antibiotic
2. vaccine


In [37]:
import requests
import json
import re

def get_primary_disease_name(disease_name):
    base_url = "https://clinicaltables.nlm.nih.gov/api/conditions/v3/search"
    # Map common disease name variations
    disease_map = {
        "covid": "COVID-19",
        "coronavirus": "COVID-19",
        "diabetes": "Diabetes mellitus",
        "obesity": "Obesity",
        "hypertension": "Hypertension"
    }
    query_name = disease_map.get(disease_name.lower(), disease_name)
    
    params = {
        "terms": query_name,
        "df": "primary_name",
        "max_list": 1
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data[0] == 0:
            print(f"No results found for '{query_name}'. Using mapped name.")
            return query_name
        
        return data[3][0][0]
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching disease data: {e}")
        return query_name

def get_treatment_names(disease_name):
    primary_name = get_primary_disease_name(disease_name)
    
    base_url = "https://api.fda.gov/drug/label.json"
    params = {
        "search": f"indications_and_usage:\"{primary_name}\"",
        "limit": 50
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        treatments = []
        if "results" in data:
            for result in data["results"]:
                drug_name = None
                if "openfda" in result:
                    if "generic_name" in result["openfda"]:
                        drug_name = result["openfda"]["generic_name"][0]
                    elif "brand_name" in result["openfda"]:
                        drug_name = result["openfda"]["brand_name"][0]
                
                if drug_name:
                    # Validate indication contains disease name
                    indications = result.get("indications_and_usage", [""])[0].lower()
                    if primary_name.lower() not in indications:
                        continue
                    # Exclude irrelevant drugs (e.g., supplements)
                    if drug_name.lower() in ["calcium carbonate", "vitamin d", "multivitamin"]:
                        continue
                    drug_name = re.sub(r"\s+", " ", drug_name.strip()).title()
                    if drug_name.lower() not in [t.lower() for t in treatments]:
                        treatments.append(drug_name)
        
        if not treatments:
            print(f"No specific treatments found for '{primary_name}' in OpenFDA drug labels.")
            return
        
        print(f"Treatments for {primary_name}:")
        for i, treatment in enumerate(sorted(treatments), 1):
            print(f"{i}. {treatment}")
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching OpenFDA data: {e}")
        print(f"No treatments available for '{primary_name}' in OpenFDA drug labels.")

# Get disease name from user
disease_name = input("Enter the disease name: ")
get_treatment_names(disease_name)

No results found for 'plauge'. Using mapped name.
Error fetching OpenFDA data: 404 Client Error: Not Found for url: https://api.fda.gov/drug/label.json?search=indications_and_usage%3A%22plauge%22&limit=50
No treatments available for 'plauge' in OpenFDA drug labels.
